In [1]:
!pip install ujson 
!pip install seaborn

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/f8/8c/5274ba7b4df814c87a8840a58e2b1dae6a489f49c3b0fad2d15f1e41d47b/ujson-5.3.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (45 kB)
You should consider upgrading via the '/opt/conda/bin/python3.8 -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/10/5b/0479d7d845b5ba410ca702ffcd7f2cd95a14a4dfff1fde2637802b258b9b/seaborn-0.11.2-py3-none-any.whl (292 kB)
You should consider upgrading via the '/opt/conda/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
import sys 
sys.path.append('../')

In [3]:
import os
from pathlib import Path
import torch
from PIL import Image
import pandas as pd 
import sys 
import json 
from src import dataset_lvis
from src import metrics
from src import helper_functions as helper
import importlib
from pathlib import Path 
from PIL import Image as PILImage
import numpy as np
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import pandas as pd 
import time 
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set()

In [4]:
importlib.reload(dataset_lvis)

<module 'src.dataset_lvis' from '/home/INM705_CW_Collins_Velagala/notebooks/../src/dataset_lvis.py'>

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Load train and validation data 

In [6]:
time_start = time.time()

data_args = {'stage': 'train',
            'classes': ['grape', 'crumb', 'pea_(food)',
                        'peach', 'apricot', 'crouton'], 
            'ds_path' : "../Datasets/coco/",
            'labels_dir': "annotations",
            'images_dir': 'images',
             'height' : 480,
             'width' : 640,
            'max_negative' : 0}
train_data = dataset_lvis.LVISData(**data_args)

print(f'Time taken to initialize train set: {time.time()-time_start}')

classes : {1: 'apricot', 2: 'crouton', 3: 'crumb', 4: 'grape', 5: 'pea_(food)', 6: 'peach'}
loaded 628 positive set images
loaded 0 negative set images
loaded 341 non-exhaustive set images
Loaded 287 images!
class 1 has 10 positive and 0 negative images
class 2 has 7 positive and 0 negative images
class 3 has 101 positive and 0 negative images
class 4 has 101 positive and 0 negative images
class 5 has 31 positive and 0 negative images
class 6 has 47 positive and 0 negative images
3283 annotations found!
stage:  train
classes:  {'apricot': 14, 'crouton': 331, 'crumb': 338, 'grape': 510, 'pea_(food)': 773, 'peach': 774}
ds_path:  ../Datasets/coco/
labels_f:  ../Datasets/coco/annotations/lvis_v1_train.json
imgs_dir:  ../Datasets/coco/images/train2017
Time taken to initialize train set: 42.40775275230408


### Split train set into custom train val set

Validation set reserved for testing

Check that we don't have too many of the rare cases in validation set - if so then there won't be much data to actually train on.

In [7]:
# what images have what classes?
g_cpu = torch.Generator()
g_cpu.manual_seed(42)

#Split dataset to test and train
indices = torch.randperm(len(train_data), generator=g_cpu).tolist()


dataset_train = torch.utils.data.Subset(train_data, indices[:260])
dataset_val = torch.utils.data.Subset(train_data, indices[260:])


class_counts = dict(zip(range(1,10), [0]*9))
for _, _, y in dataset_val:
    for label in y['labels']:
        class_counts[label.item()] += 1
            
class_counts

{1: 0, 2: 0, 3: 55, 4: 198, 5: 79, 6: 27, 7: 0, 8: 0, 9: 0}

## Prepare test set

The LVIS validation set is reserved for our test set

In [8]:
time_start = time.time() 
data_args = {'stage': 'val',
            'classes': ['grape', 'crumb', 'pea_(food)',
                        'peach', 'apricot', 'crouton'], 
            'ds_path' : "../Datasets/coco/",
            'labels_dir': "annotations",
            'images_dir': 'images',
             'height' : 480,
             'width' : 640,
            'max_negative' : 200}   # note that we include negative sets for the testing
test_data = dataset_lvis.LVISData(**data_args)
print(f'Time taken to initialize val set: {time.time()-time_start}')

classes : {1: 'apricot', 2: 'crouton', 3: 'crumb', 4: 'grape', 5: 'pea_(food)', 6: 'peach'}
loaded 117 positive set images
loaded 887 negative set images
loaded 59 non-exhaustive set images
Loaded 942 images!
class 1 has 0 positive and 119 negative images
class 2 has 3 positive and 140 negative images
class 3 has 18 positive and 124 negative images
class 4 has 22 positive and 174 negative images
class 5 has 7 positive and 181 negative images
class 6 has 9 positive and 169 negative images
791 annotations found!
stage:  val
classes:  {'apricot': 14, 'crouton': 331, 'crumb': 338, 'grape': 510, 'pea_(food)': 773, 'peach': 774}
ds_path:  ../Datasets/coco/
labels_f:  ../Datasets/coco/annotations/lvis_v1_val.json
imgs_dir:  ../Datasets/coco/images/train2017
Time taken to initialize val set: 4.068005561828613


# Fine-tuning the model

Here we set `pretrained = False` for the FPN: FPN is pre-trained on coco dataset so we'll try to retrain from scratch. We keep the backbone ResNet50 pretrained on ImageNet.

In [9]:
      
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

# Set up DataLoaders

In [10]:

train_loader = torch.utils.data.DataLoader(
 dataset_train, batch_size=4, shuffle=True, #num_workers=4,
 collate_fn=helper.CollateCustom())

val_loader = torch.utils.data.DataLoader(
 dataset_val, batch_size=4, shuffle=True, #num_workers=4,
 collate_fn=helper.CollateCustom())


# Initialize Model + Model params

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has 10 classes - background and 9 specified objects
num_classes = 10

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Train Model

In [ ]:
num_epochs = 31
time_start = time.time() 


train_loss_df = pd.DataFrame(columns = ['epoch', 'loss_classifier', 'loss_box_reg', 'loss_mask', 'loss_objectness', 'loss_rpn_box_reg', 'total_loss'])
val_loss_df = pd.DataFrame(columns = ['epoch', 'loss_classifier', 'loss_box_reg', 'loss_mask', 'loss_objectness', 'loss_rpn_box_reg', 'total_loss'])

loss_types = ['loss_classifier', 'loss_box_reg', 'loss_mask', 'loss_objectness', 'loss_rpn_box_reg', 'total_loss']


for epoch in range(num_epochs):
    model.train()
    
    train_loss = dict.fromkeys(loss_types, 0) 
    val_loss = dict.fromkeys(loss_types, 0) 
    
    """
    Train 
    """
    for batch_num, (idx, X, y) in enumerate(train_loader):
        #print(idx)
        X = X.to(device)
        y = [{k: v.to(device) for k, v in t.items()} for t in y]
        
        loss_dict = model(X, y) 
        
        losses = sum(loss for loss in loss_dict.values())
        
        #save losses
        for key in loss_types:
            if key != 'total_loss':
                train_loss[key] += loss_dict[key].item()
            else: 
                train_loss['total_loss'] += losses.item()
                

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
    row = pd.DataFrame({'epoch': [epoch],
          'loss_classifier': [train_loss['loss_classifier']/(batch_num+1)],
          'loss_box_reg' : [train_loss['loss_box_reg']/(batch_num+1)],
           'loss_mask': [train_loss['loss_mask']/(batch_num+1)],
           'loss_objectness': [train_loss['loss_objectness']/(batch_num+1)],
           'loss_rpn_box_reg': [train_loss['loss_rpn_box_reg']/(batch_num+1)],
            'total_loss': [train_loss['total_loss']/(batch_num+1)] 
          })     

    train_loss_df = pd.concat([train_loss_df, row], ignore_index = True, axis = 0)
    
    print(f"Training loss after {epoch} epochs: {train_loss['total_loss']}")

    
    """
    Validation
    """
    with torch.no_grad():
        for batch_num, (idx, X, y) in enumerate(val_loader):
            X = X.to(device)
            y = [{k: v.to(device) for k, v in t.items()} for t in y]

            loss_dict = model(X, y) 
            
            losses_val = sum(loss for loss in loss_dict.values())
            
            #save losses
            for key in loss_types:
                if key != 'total_loss':
                    val_loss[key] += loss_dict[key].item()
                else: 
                    val_loss['total_loss'] += losses_val.item()
                    
        row = pd.DataFrame({'epoch': [epoch],
                          'loss_classifier': [val_loss['loss_classifier']/(batch_num+1)],
                          'loss_box_reg' : [val_loss['loss_box_reg']/(batch_num+1)],
                           'loss_mask': [val_loss['loss_mask']/(batch_num+1)],
                           'loss_objectness': [val_loss['loss_objectness']/(batch_num+1)],
                           'loss_rpn_box_reg': [val_loss['loss_rpn_box_reg']/(batch_num+1)],
                            'total_loss': [val_loss['total_loss']/(batch_num+1)] 
                          })
        val_loss_df = pd.concat([val_loss_df, row], ignore_index = True, axis = 0)

    print(f"Validation loss after {epoch} epochs: {val_loss['total_loss']}") 
    print(f'Time elapsed for {epoch} epochs: {time.time()-time_start}') 

      
    """
    Save checkpoints and losses every 5 epoch
    """
    if epoch%5 == 0: 
        checkpoint = {"state_dict": model.state_dict(),
                        "optimizer": optimizer.state_dict(),
                        "step": epoch,
                        "ds_args" : data_args
                        }
        fname = "model_" + str(epoch) + "_epochs.pth"
        helper.save_checkpoint(checkpoint, fname)
        print(f'Time elapsed after {epoch} epochs: {time.time()-time_start}')  
        val_loss_df.to_csv(Path.cwd().parent.joinpath("val_loss.csv"))
        train_loss_df.to_csv(Path.cwd().parent.joinpath("train_loss.csv"))

    
    
    #for final epoch 
    if epoch == num_epochs-1: 
        helper.save_model(model.state_dict(), "model.pth")
        val_loss_df.to_csv(Path.cwd().parent.joinpath("val_loss.csv"))
        train_loss_df.to_csv(Path.cwd().parent.joinpath("train_loss.csv"))
    
    
    
print(f"Time elapsed for {epoch+1} epochs: {round((time.time()-time_start)/60, 2)} min")

Training loss after 0 epochs: 92.93645197153091
Validation loss after 0 epochs: 8.893099248409271
Time elapsed for 0 epochs: 47.035053730010986
Saved checkpoint model_0_epochs.pth!
Time elapsed after 0 epochs: 47.53584337234497
Training loss after 1 epochs: 70.51079231500626
Validation loss after 1 epochs: 8.07829761505127
Time elapsed for 1 epochs: 95.66369438171387
Training loss after 2 epochs: 61.59013074636459
Validation loss after 2 epochs: 8.422475576400757
Time elapsed for 2 epochs: 143.78244352340698
Training loss after 3 epochs: 55.91294676065445
Validation loss after 3 epochs: 9.219867527484894
Time elapsed for 3 epochs: 192.26619291305542
Training loss after 4 epochs: 52.95373582839966
Validation loss after 4 epochs: 8.494624316692352
Time elapsed for 4 epochs: 240.8719892501831
Training loss after 5 epochs: 50.05961659550667
Validation loss after 5 epochs: 8.63841313123703
Time elapsed for 5 epochs: 289.78059673309326
Saved checkpoint model_5_epochs.pth!
Time elapsed after 

# Plot loss 

In [ ]:
new_df = pd.DataFrame() 
#new_df['epoch'] = train_loss_df['epoch'] 
new_df['train_loss'] = train_loss_df['total_loss'] 
new_df['val_loss'] = val_loss_df['total_loss'] 

sns.lineplot(data = new_df[1:])
new_df


In [ ]:
train_loss_df

In [ ]:
val_loss_df

## Run the testing

In [12]:
importlib.reload(metrics)
importlib.reload(dataset_lvis)

<module 'src.dataset_lvis' from '/home/INM705_CW_Collins_Velagala/notebooks/../src/dataset_lvis.py'>

In [13]:
test_loader = torch.utils.data.DataLoader(
 test_data, batch_size=1, shuffle=True, #num_workers=4,
 collate_fn=helper.CollateCustom())

If testing at different time from training, then load checkpoints.

Or load checkpoints from earlier stopping point

In [14]:
cp_path = Path('../model_checkpoints/model_30_epochs.pth')
helper.load_checkpoint(cp_path, model, optimizer)


Loaded checkpoint ../model_checkpoints/model_30_epochs.pth!


In [15]:
%%time

pred_boxes = []
gt = []
for idx, X, y in test_loader:
    model.eval()
    y_pred = model(X.to(device))
    # keep running list of predictions and ground truths
    pred_boxes, gt = metrics.store_preds(idx, y, y_pred,
                                     pred_boxes, gt)

mAP_list = []
thresholds = list(np.around(np.arange(0.5, 1.0, 0.05),2))
for iou_thresh in thresholds:
    mAP_list.append(metrics.calculate_ap(pred_boxes, gt, 
                               iou_threshold=iou_thresh,
                               class_datasets = test_data.class_datasets))

for i in range(len(thresholds)):
    print('mAP for iou_threshold of {}: {:.3f}'.format(thresholds[i], mAP_list[i][0]))

/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1153.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


mAP for iou_threshold of 0.5: 0.185
mAP for iou_threshold of 0.55: 0.178
mAP for iou_threshold of 0.6: 0.170
mAP for iou_threshold of 0.65: 0.157
mAP for iou_threshold of 0.7: 0.134
mAP for iou_threshold of 0.75: 0.112
mAP for iou_threshold of 0.8: 0.079
mAP for iou_threshold of 0.85: 0.041
mAP for iou_threshold of 0.9: 0.013
mAP for iou_threshold of 0.95: 0.000
CPU times: user 9min 40s, sys: 17.9 s, total: 9min 58s
Wall time: 1min 34s


### Print out per category mAP

Table below shows the categories we have in rows, and different IoU thresholds in columns.

We also print the number of images in the test set containing each category. You can see that the model doesn't have much chance to prove itself on chessboard or drumstick categories!

In [18]:
df = pd.DataFrame(index = list(range(1, 7)), columns = ['positives'] + thresholds)

df['positives'] = [len(c['positive']) for c in test_data.class_datasets.values()]

for i in range(len(thresholds)):
    data = [v.item() for v in mAP_list[i][1].values()]
    df[thresholds[i]] = data

df.rename(index=test_data.idx_to_classname, inplace=True)
# remove apricot as no data
df = df.iloc[1:,:]
df.loc['frequent_mean'] = df[df.positives > 15].mean()
df.loc['common_mean'] = df[(df.positives < 15) & (df.positives > 5)].mean()
df.loc['rare_mean'] = df[df.positives < 5].mean()
df['class_mAP'] = df.iloc[:, 1:].mean(axis=1)
df.round(4)

,positives,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,class_mAP
crouton,3.0,0.0103,0.0103,0.0103,0.0103,0.0103,0.0103,0.0066,0.0066,0.0000,0.0000,0.0075
crumb,18.0,0.1545,0.1481,0.1203,0.0873,0.0697,0.0493,0.0157,0.0006,0.0001,0.0000,0.0646
grape,22.0,0.3950,0.3819,0.3721,0.3514,0.3306,0.2867,0.2354,0.1399,0.0462,0.0023,0.2542
pea_(food),7.0,0.3656,0.3407,0.3304,0.3088,0.2403,0.1779,0.0865,0.0332,0.0020,0.0000,0.1885
peach,9.0,0.1848,0.1848,0.1848,0.1814,0.1529,0.1483,0.1309,0.0659,0.0301,0.0003,0.1264
frequent_mean,20.0,0.2748,0.2650,0.2462,0.2193,0.2002,0.1680,0.1256,0.0703,0.0232,0.0011,0.1594
common_mean,8.0,0.2752,0.2627,0.2576,0.2451,0.1966,0.1631,0.1087,0.0496,0.0160,0.0002,0.1575
rare_mean,3.0,0.0103,0.0103,0.0103,0.0103,0.0103,0.0103,0.0066,0.0066,0.0000,0.0000,0.0075
